In [1]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import ast

In [2]:
from collections import Counter

def get_rare_tokens(df: pd.DataFrame, threshold: int=2) -> set:
    all_sentences = df['tokens'].tolist()
    all_tokens = [token for sentence in all_sentences for token in sentence]
    token_counts = Counter(all_tokens)
    rare_tokens = {token for token, count in token_counts.items() if count < threshold}
    return rare_tokens

# Predict on the original splits

In [3]:
df = pd.read_csv("data/preprocessed_data.csv")
df['tokens'] = df['tokens'].apply(ast.literal_eval)

df_train = df[df['dataset'] == 'train']
df_train.head()

,sentence,topic,sentiment,dataset,tokens
0,slide giáo trình đầy đủ .,1,2,train,"[slide, giáo_trình, đầy_đủ]"
1,"nhiệt tình giảng dạy , gần gũi với sinh viên .",0,2,train,"[nhiệt_tình, giảng_dạy, gần_gũi, sinh_viên]"
2,đi học đầy đủ full điểm chuyên cần .,1,0,train,"[đi, học, đầy_đủ, full, chuyên_cần]"
3,chưa áp dụng công nghệ thông tin và các thiết ...,0,0,train,"[áp_dụng, công_nghệ, thông_tin, thiết_bị, giản..."
4,"thầy giảng bài hay , có nhiều bài tập ví dụ ng...",0,2,train,"[thầy, giảng, bài_tập, ví_dụ, lớp]"


In [4]:
rare_tokens = get_rare_tokens(df_train, 2)

print(f"The number of rare tokens is: {len(rare_tokens)}")
print(f"Some rare tokens are: {list(rare_tokens)[:5]}")

The number of rare tokens is: 1623
Some rare tokens are: ['dễ_dãi', 'lãnh_đạm', 'rối_ren', 'lặp_if', 'cứng_rắn']


In [5]:
df['popular'] = df['tokens'].apply(lambda x: ['<UNK>' if token in rare_tokens else token for token in x ])
df.head()

,sentence,topic,sentiment,dataset,tokens,popular
0,slide giáo trình đầy đủ .,1,2,train,"[slide, giáo_trình, đầy_đủ]","[slide, giáo_trình, đầy_đủ]"
1,"nhiệt tình giảng dạy , gần gũi với sinh viên .",0,2,train,"[nhiệt_tình, giảng_dạy, gần_gũi, sinh_viên]","[nhiệt_tình, giảng_dạy, gần_gũi, sinh_viên]"
2,đi học đầy đủ full điểm chuyên cần .,1,0,train,"[đi, học, đầy_đủ, full, chuyên_cần]","[đi, học, đầy_đủ, <UNK>, chuyên_cần]"
3,chưa áp dụng công nghệ thông tin và các thiết ...,0,0,train,"[áp_dụng, công_nghệ, thông_tin, thiết_bị, giản...","[áp_dụng, công_nghệ, thông_tin, thiết_bị, giản..."
4,"thầy giảng bài hay , có nhiều bài tập ví dụ ng...",0,2,train,"[thầy, giảng, bài_tập, ví_dụ, lớp]","[thầy, giảng, bài_tập, ví_dụ, lớp]"


In [ ]:
select_cols = ['popular', 'topic', 'sentiment']

df_train = df[df['dataset'] == 'train'][select_cols]
df_test = df[df['dataset'] == 'test'][select_cols]

In [ ]:
train_sentences = [" ".join(sent) for sent in df_train['popular']]
test_sentences = [" ".join(sent) for sent in df_test['popular']]

In [15]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(train_sentences).toarray()
X_valid = vectorizer.transform(valid_sentences).toarray()
X_test = vectorizer.transform(test_sentences).toarray()

## Predict topics

In [16]:
y_train = df_train['topic']
y_valid = df_valid['topic']
y_test = df_test['topic']

In [17]:
param_grids = {
    'GaussianNB': {
        'var_smoothing': [1e-7, 1e-6, 1e-5]
    },
    'SVC': {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf']
    },
    'RandomForest': {
        'n_estimators': 100,
        'max_depth': [2, 3, 4]
    }
}

In [18]:
best_models = {}
best_params = {}
best_scores = {}

In [20]:
import itertools
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

for name, grid in param_grids.items():
    ModelClass = {
        'GaussianNB': GaussianNB,
        'SVC': SVC,
        'RandomForest': RandomForestClassifier
    }[name]
    
    best_f1 = -1
    best_model = None
    best_param = None
    
    # iterate over all combinations of hyperparameters
    keys, values = zip(*grid.items())
    for vals in itertools.product(*values):
        params = dict(zip(keys, vals))
        # ensure reproducibility for SVC and RF
        if name in ('SVC', 'RandomForest'):
            params['random_state'] = 42
        
        model = ModelClass(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_valid)
        f1 = f1_score(y_valid, y_pred, average='macro', zero_division=0)
        
        if f1 > best_f1:
            best_f1 = f1
            best_model = model
            best_param = params
    
    best_models[name] = best_model
    best_params[name] = best_param
    best_scores[name] = best_f1
    print(f"{name} → Best params: {best_param}, F1 = {best_f1:.4f}")

GaussianNB → Best params: {'var_smoothing': 1e-05}, F1 = 0.2727


KeyboardInterrupt: 